In [1]:
import websocket
import json
import pandas as pd
import numpy as np
from datetime import datetime
from dataUtils import *

In [2]:
import keras
model = keras.models.load_model("DNNMODEL")
# Loading mu and std
import pickle
params = pickle.load(open("params.pkl", "rb"))
mu = params["mu"]
std = params["std"]

In [3]:
dtypes = np.dtype([
    ('date', str),
    ('open', float),
    ('high', float),
    ('low', float),
    ('close', float),
    ('VolETH', float),
    ('Returns', float),
    ('Direction', float),
    ('SMA', float),
    ('EMA', float),
    ('Min', float),
    ('Max', float),
    ('RSI', float),
    ('UpperBB', float),
    ('LowerBB', float),
    ])
data = np.empty(0, dtype=dtypes)
df = pd.DataFrame(data)
df.set_index(["date"], inplace=True)

In [4]:
df.head(10)

,open,high,low,close,VolETH,Returns,Direction,SMA,EMA,Min,Max,RSI,UpperBB,LowerBB
date,,,,,,,,,,,,,,


In [5]:
def createNewFeatures(t):
    if len(df.index)<2:
        return

    df.loc[t, 'Returns'] = np.log(df.iloc[-1]['close']/ df.iloc[-2]['close'])
    df.loc[t, 'Direction'] = 1 if df.loc[t]['Returns'] > 0 else 0
    
    window = 5

    if len(df.index)>30:
        window=20
    
    if len(df.index)>window:
        df.loc[t, 'SMA'] = df.tail(window)['close'].mean()
        df['EMA'] = df['close'].ewm(span=20, adjust=False).mean()
        df['Min'] = df.tail(window)['close'].min() / df['close'] - 1
        df['Max'] = df.tail(window)['close'].max() / df['close'] - 1
        df['RSI'] = get_RSI(df['close'])
        SD = df.tail(window)['close'].std()
        df.loc[t, 'UpperBB'] = df.loc[t]['SMA'] + (2 * SD)
        df.loc[t, 'LowerBB'] = df.loc[t]['SMA'] - (2 * SD)

In [6]:
def updateData(price, t, vol):
    open, high, low, close, volume = 0,0,0,0,0
    global df
    if t in df.index:
        open = df.loc[t]['open']
        high = df.loc[t]['high']
        low = df.loc[t]['low']
        close = price
        volume = df.loc[t]['VolETH'] + vol
    else:
        open = price
        high = price
        low = price
        close = price
        volume = vol
        df = df.append(pd.Series(name=t, dtype=int))
    

    if price<low:
        low = price
    if price>high:
        high = price
    
    df.loc[t, 'open'] = open
    df.loc[t, 'high'] = high
    df.loc[t, 'low'] = low
    df.loc[t, 'close'] = close
    df.loc[t, 'VolETH'] = volume
#updateData()


In [7]:
def createNewLags(lags, time):
    cols = ['open', 'high', 'low', 'close', 'VolETH']
    features = ['Direction', 'SMA', 'EMA', 'Min', 'Max', 'RSI', 'UpperBB', 'LowerBB']
    for f in features:
        for lag in range(1, lags+1):
            col = "{}_lag_{}".format(f, lag)
            df.loc[time, col] = df.iloc[-1-lag][f]
            cols.append(col)
    return cols    

In [8]:
def predict(cols, time):
    df_s = (df - mu) / std
    pred = model.predict(df_s[cols])
    print(pred[-1])

In [9]:
def on_message(ws, message):
    dict = json.loads(message)
    price = dict['data'][0]['p']
    time = datetime.fromtimestamp(int(dict['data'][0]['t']/1000))
    time = time.strftime("%D %H:%M")
    vol = dict['data'][0]['v']
    updateData(price, time, vol)
    createNewFeatures(time)
    cols = createNewLags(5, time)
    predict(cols, time)
    

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("### closed ###")

def on_open(ws):
    ws.send('{"type":"subscribe","symbol":"BINANCE:ETHUSDT"}')


In [10]:
ws = websocket.WebSocketApp("wss://ws.finnhub.io?token=c23hkuqad3ieeb1ld9jg",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)

In [ ]:
ws.on_open = on_open
ws.run_forever()